# Importing various libraries

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import _pickle as cPickle
from scipy.io import loadmat
from sklearn.svm import SVC
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve
from sklearn.naive_bayes import MultinomialNB

ModuleNotFoundError: No module named 'textblob'

In [23]:
df=pd.read_csv("sample.csv");
df.head(3)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O


This dataset is downloaded from https://www.kaggle.com/c/si650winter11/data
this is a TSV ("tab separated values") file, where the first column is a label saying whether the given review
is positive or negative. The second column is the review itself.
Data is tab separeted and therefore "\t" is passed as separator parameter to function.

In [24]:
print(len(df))

9989


Total no of reviews.

In [25]:
df.groupby('Sentiment').describe()

ItemID                                                           \
            count         mean          std  min      25%     50%      75%   
Sentiment                                                                    
0          5807.0  4381.127260  2730.160881  1.0  2086.50  4016.0  6436.50   
1          4182.0  5852.401243  2879.241444  3.0  3772.25  6458.5  8239.25   

                    
               max  
Sentiment           
0          10000.0  
1           9999.0

# Data preprocessing

In [26]:
def tokens(review):
    return TextBlob(review).words

In [27]:
df.head().SentimentText.apply(tokens)

0                  [is, so, sad, for, my, APL, friend]
1                 [I, missed, the, New, Moon, trailer]
2                         [omg, its, already, 7:30, O]
3    [Omgaga, Im, sooo, im, gunna, CRy, I, 've, bee...
4        [i, think, mi, bf, is, cheating, on, me, T_T]
Name: SentimentText, dtype: object

Function tokens() is created to parse data/review into words.

In [28]:
TextBlob("ready was not a good movie").tags
#nltk.help.upenn_tagset('JJ')

[('ready', 'NN'),
 ('was', 'VBD'),
 ('not', 'RB'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('movie', 'NN')]

.tags is inbuilt function provided by TextBlob it is used to assign part of speech tags to the words in text.
It gives list of (word, POS) pairs.
To check meaning of particular tag nltk.help.upenn_tagset('tagname') can be used eg. nltk.help.upenn_tagset('JJ')

In [29]:
nltk.download("wordnet", "C:/nltk_data/")
def to_lemmas(review):
    wordss = TextBlob(review.lower()).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in wordss]

df.SentimentText.head().apply(to_lemmas)

[nltk_data] Downloading package wordnet to C:/nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!


0                  [is, so, sad, for, my, apl, friend]
1                 [i, missed, the, new, moon, trailer]
2                          [omg, it, already, 7:30, o]
3    [omgaga, im, sooo, im, gunna, cry, i, 've, bee...
4        [i, think, mi, bf, is, cheating, on, me, t_t]
Name: SentimentText, dtype: object

Lemmatization is one of the important satge of data preprocessing in this step words are converted to their lemma(base form). For example "octopi" is converted to "octopus". similar method is stemming.

NLTK also provide very powerful lemmatizer which make use of WORDNET eg.

In [30]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('octopi')
#nltk

'octopus'

# Converting text data into vectors 

In [32]:
bow_transformer = CountVectorizer(analyzer=to_lemmas).fit(df['SentimentText'])
print(len(bow_transformer.vocabulary_))

17430


Sklearn offers Countvectorizer which counts the frequency of particular word in document. This assigns a unique number to every word in collection.

In [33]:
review1=df['SentimentText'][3]
print(review1)
#to check 3rd document/review in collection/database

          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...


In [34]:
bow=bow_transformer.transform([review1])
print(bow)
bow.shape

  (0, 82)	1
  (0, 158)	1
  (0, 255)	1
  (0, 382)	1
  (0, 668)	1
  (0, 1495)	1
  (0, 1936)	1
  (0, 4041)	1
  (0, 4060)	1
  (0, 4458)	1
  (0, 6476)	1
  (0, 6889)	1
  (0, 7570)	2
  (0, 7697)	2
  (0, 8444)	1
  (0, 10978)	1
  (0, 10983)	1
  (0, 12150)	1
  (0, 13620)	1
  (0, 13927)	1
  (0, 14515)	1
  (0, 15049)	1
  (0, 16414)	1


(1, 17430)

Countvectorizer creates sparse matrix.

In [36]:
print(bow_transformer.get_feature_names_out()[372])
#to check 372nd word in collection

3.14


In [38]:
review_bow = bow_transformer.transform(df['SentimentText'])
print( 'sparse matrix shape:', review_bow.shape)
print('number of non-zeros:', review_bow.nnz) #learn this
print( 'sparsity: %.2f%%' % (100.0 * review_bow.nnz))

sparse matrix shape: (9989, 17430)
number of non-zeros: 120246
sparsity: 12024600.00%


shape of sparse matrix n*m where n are total documents and m are total unique words.

CountVectorizer just count the frequency of word in that document. But many time few words such as(the, or) occurs a lot of time in collection which really don't contribute in deciding the polarity of particular document so to nullify their effect special weighting method is to be used. 

# Tf-idf Vectorizer

Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency.

The goal of using tf-idf instead of the just CountVectorizer in a given document is to scale down the impact of tokens that occur very frequently in a given corpus(which are less informative) than tokens which occur few times.

Tf-idf(d,t)=tf(d,t)*idf(d,t);

where tf(d,t) is term frequency which states how many times word/token t occur in that doucument devided by total no of words in that document.

and idf(d,t)=log[n/(df(d,t))] i.e. total no of documents divided by no of documents containing that word/token t.

In [39]:
tfidf_transformer =TfidfTransformer().fit(review_bow)
review_tfidf = tfidf_transformer.transform(review_bow)
review_tfidf.shape

(9989, 17430)

Count vectorizer gives output as frequency of diiferent words in our corpus this is then passed to transform method of tf-idf_transformer.

This Transform a count matrix to a normalized tf or tf-idf representation



In [41]:
text_train, text_test, liked_train, liked_test = train_test_split(df['SentimentText'], df['Sentiment'], test_size=0.2)
print(len(text_train), len(text_test), len(text_train) , len(text_test))


7991 1998 7991 1998


The dataset downloaded is then divided into training data and test data with ratio 0.8 to 0.2

In [42]:
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=to_lemmas)),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC()),
])

Pipeline sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit.

The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.

In [43]:
# pipeline parameters to automatically explore and tune
param_svm = [
  {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
  {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]


SVM

In [48]:
grid_svm = GridSearchCV(
    pipeline_svm, #object used to fit the data
    param_grid=param_svm, 
    refit=True,  # fit using all data, on the best detected classifier
    n_jobs=-1,  # number of cores to use for parallelization; -1 for "all cores" i.e. to run on all CPUs
    scoring='accuracy',#optimizing parameter
    cv=StratifiedKFold(n_splits=5),
)

Exhaustive search over specified parameter values for an estimator.

CV stands for cross validations. Learning the parameters of a prediction function and testing it on the same data is a methodological mistake, it will always give 100% accuracy and therefore training and testing data must be different. Cross validation is idea of dividing training data into k folds i.e. k subset. The following procedure is followed for each of the k “folds”:

A model is trained using k-1 of the folds as training data;

The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

  

In [ ]:
%time classifier = grid_svm.fit(text_train, liked_train) # find the best combination from param_svm

In [50]:
print(classifier.cv_results_)

{'mean_fit_time': array([ 9.13037014, 11.08522954, 12.18450933, 10.59441023,  8.44368329,
        8.01027851,  8.05471473,  8.13315444,  7.57888122,  8.13708167,
        7.85991654,  6.84219036]), 'std_fit_time': array([0.26407811, 1.29783626, 0.34860532, 0.83409847, 0.4207121 ,
       0.35123636, 0.29753969, 0.30256759, 0.2641206 , 0.36101811,
       0.28899139, 0.5939673 ]), 'mean_score_time': array([1.4549139 , 1.42496543, 1.36910787, 1.21375551, 1.74703059,
       1.81991067, 1.78848014, 1.82378883, 1.66193604, 1.83337965,
       1.53626881, 1.28753438]), 'std_score_time': array([0.06156939, 0.08474733, 0.03126889, 0.05723953, 0.01704184,
       0.05255064, 0.05477495, 0.06909747, 0.03155687, 0.06796363,
       0.07315922, 0.08859057]), 'param_classifier__C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False],
       fill_value='?',
      

In [51]:
print(classification_report(liked_test, classifier.predict(text_test)))

              precision    recall  f1-score   support

           0       0.79      0.86      0.82      1163
           1       0.78      0.69      0.73       835

    accuracy                           0.79      1998
   macro avg       0.78      0.77      0.78      1998
weighted avg       0.79      0.79      0.78      1998



In [52]:
print(classifier.predict(["the vinci code is awesome"])[0])

1


In [53]:
print(classifier.predict(["the vinci code is bad"])[0])

0


In [54]:
def gaussKernel(x1, x2, sigma):
    ss=np.power(sigma,2)
    norm= (x1-x2).T.dot(x1-x2)
    return np.exp(-norm/(2*ss))
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2
gaussKernel(x1,x2,sigma)

0.32465246735834974